# Ajuste de Parâmetros e Simulações
### versão 3
17/08/2021

Update para refazer as figuras a pedido dos revisores de JTB

## Objetivos
1. Ajustar os dados do índice de isolamento a través de um polinômio
2. Utilizar $\theta$ vomo função do tempo
3. Ajustar o modelo sem vacinação com $\theta$ variável
4. Ajustar os parâmetros do modelo Covid sem vacinação

In [1]:
# Carregando librarias
# from sympy import transpose as tp
import numpy as np
import pandas as pd
#from sklearn.linear_model import LinearRegression
#from sklearn.preprocessing import PolynomialFeatures
from numpy.polynomial import polynomial as poly
import scipy.optimize as optimization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from myfunctions import rk4
from scipy.integrate import odeint
from matplotlib.dates import MonthLocator, YearLocator
# funções 
def mm_to_inch(value):
    return value/(2.54*10)
# tamanho de todas as figuras
fig_width = mm_to_inch(140)
fig_height = mm_to_inch(140)
fig_dpi = 300
#plt.rcParams["figure.figsize"] = (mm_to_inch(140),mm_to_inch(140))
#plt.rcParams["figure.dpi"] = 300

# Carregando dados do Índice de isolamento

## Definir data máxima de coleta de dados

In [2]:
data_max = '2021-02-28'
print('Dados coletados até: ',data_max)

Dados coletados até:  2021-02-28


In [3]:
# carregando data do índice de isolamento
saopaulo_isol_data = pd.read_csv("data/SaoPaulo_isolamento.csv")
saopaulo_isol_df = pd.DataFrame(saopaulo_isol_data)
saopaulo_isol_df.drop(labels='Unnamed: 0', axis=1, inplace=True)
saopaulo_isol_df = saopaulo_isol_df.rename(columns={'Isol':'Dados'})
saopaulo_isol_df = saopaulo_isol_df[(saopaulo_isol_df['Data']<=data_max)]
saopaulo_isol_df['Data'] = pd.to_datetime(saopaulo_isol_df['Data'], yearfirst=True)
print("Resumo Dados de isolamento - São Paulo")
saopaulo_isol_data.describe()

Resumo Dados de isolamento - São Paulo


,Unnamed: 0,Isol
count,372.000000,372.000000
mean,932.500000,0.443925
std,107.531391,0.058537
min,747.000000,0.250000
25%,839.750000,0.400000
50%,932.500000,0.430000
75%,1025.250000,0.480000
max,1118.000000,0.590000


In [4]:
# carregando data do índice de isolamento
campinas_isol_data = pd.read_csv("data/Campinas_isolamento.csv")
campinas_isol_df = pd.DataFrame(campinas_isol_data)
campinas_isol_df.drop(labels='Unnamed: 0', axis=1, inplace=True)
campinas_isol_df = campinas_isol_df.rename(columns={'Isol':'Dados'})
campinas_isol_df = campinas_isol_df[(campinas_isol_df['Data']<=data_max)]
campinas_isol_df['Data'] = pd.to_datetime(campinas_isol_df['Data'], yearfirst=True)
print("Resumo Dados de isolamento - Campinas")
campinas_isol_data.describe()

Resumo Dados de isolamento - Campinas


,Unnamed: 0,Isol
count,373.000000,373.000000
mean,187.000000,0.414021
std,107.820066,0.065010
min,1.000000,0.270000
25%,94.000000,0.360000
50%,187.000000,0.400000
75%,280.000000,0.460000
max,373.000000,0.590000


In [5]:
# carregando data do índice de isolamento
santos_isol_data = pd.read_csv("data/Santos_isolamento.csv")
santos_isol_df = pd.DataFrame(santos_isol_data)
santos_isol_df.drop(labels='Unnamed: 0', axis=1, inplace=True)
santos_isol_df = santos_isol_df.rename(columns={'Isol':'Dados'})
santos_isol_df = santos_isol_df[(santos_isol_df['Data']<=data_max)]
santos_isol_df['Data'] = pd.to_datetime(santos_isol_df['Data'], yearfirst=True)
print("Resumo Dados de isolamento - Santos")
santos_isol_data.describe()

Resumo Dados de isolamento - Santos


,Unnamed: 0,Isol
count,373.000000,373.000000
mean,560.000000,0.414665
std,107.820066,0.063608
min,374.000000,0.260000
25%,467.000000,0.360000
50%,560.000000,0.390000
75%,653.000000,0.460000
max,746.000000,0.590000


# Carregando Dados públicos do Covid-19
Obtidos do site: https://www.seade.gov.br/coronavirus/

In [6]:
# Carregando os dados para São Paulo
saopaulo_covid_df = pd.read_csv("data/SaoPaulo_dados_covid.csv")
saopaulo_covid_df.drop(labels='Unnamed: 0', axis=1, inplace=True)
saopaulo_covid_df = saopaulo_covid_df[(saopaulo_covid_df['datahora']<=data_max)]
# salvar df truncada para arquivo .csv 
saopaulo_covid_df.to_csv('SaoPaulo_dados_covid_trunc.csv', index=False)
#
saopaulo_covid_casos = saopaulo_covid_df['casos'].to_numpy()
# população total de São Paulo
saopaulo_tot_pop = int(saopaulo_covid_df['pop'].mean())
# vetor dos dias registrados 
saopaulo_dias = pd.to_datetime(saopaulo_covid_df['datahora'], yearfirst=True)
# vetor 0, 1, ..., N do tamanho das amostras coletadas
saopaulo_tempo = np.arange(0, saopaulo_covid_casos.size, 1)
# número de amostras
N_saopaulo = saopaulo_tempo.size
#
print("Dados de casos confirmados - São Paulo")
saopaulo_covid_df.describe()

Dados de casos confirmados - São Paulo


,casos,obitos,pop
count,370.000000,370.000000,370.0
mean,224968.448649,9592.770270,11869660.0
std,163400.820040,6004.558403,0.0
min,1.000000,0.000000,11869660.0
25%,50127.750000,3859.000000,11869660.0
50%,255328.500000,11311.000000,11869660.0
75%,348681.000000,14373.000000,11869660.0
max,524858.000000,18642.000000,11869660.0


In [7]:
# Carregando os dados para Campinas
campinas_covid_df = pd.read_csv("data/Campinas_dados_covid.csv")
campinas_covid_df.drop(labels='Unnamed: 0', axis=1, inplace=True)
campinas_covid_df = campinas_covid_df[(campinas_covid_df['datahora']<=data_max)]
# salvar df truncada para arquivo .csv 
campinas_covid_df.to_csv('Campinas_dados_covid_trunc.csv', index=False)
#
campinas_covid_casos = campinas_covid_df['casos'].to_numpy()
# população total de Campinas
campinas_tot_pop = int(campinas_covid_df['pop'].mean())
# vetor dos dias registrados 
campinas_dias = pd.to_datetime(campinas_covid_df['datahora'], yearfirst=True)
# vetor 0, 1, ..., N do tamanho das amostras coletadas
campinas_tempo = np.arange(0, campinas_covid_casos.size, 1)
# número de amostras
N_campinas = campinas_tempo.size
#
print("Dados de casos confirmados - Campinas")
campinas_covid_df.describe()

Dados de casos confirmados - Campinas


,casos,obitos,pop
count,370.000000,370.000000,370.0
mean,22967.845946,826.535135,1175501.0
std,18762.603145,638.785212,0.0
min,0.000000,0.000000,1175501.0
25%,1470.500000,64.250000,1175501.0
50%,26619.500000,997.500000,1175501.0
75%,36654.000000,1371.250000,1175501.0
max,58250.000000,1869.000000,1175501.0


In [8]:
# Carregando os dados para Santos
santos_covid_df = pd.read_csv("data/Santos_dados_covid.csv")
santos_covid_df.drop(labels='Unnamed: 0', axis=1, inplace=True)
santos_covid_df = santos_covid_df[(santos_covid_df['datahora']<=data_max)]
# salvar df truncada para arquivo .csv 
santos_covid_df.to_csv('Santos_dados_covid_trunc.csv', index=False)
#
santos_covid_casos = santos_covid_df['casos'].to_numpy()
# população total de Santos
santos_tot_pop = int(santos_covid_df['pop'].mean())
# vetor dos dias registrados 
santos_dias = pd.to_datetime(santos_covid_df['datahora'], yearfirst=True)
# vetor 0, 1, ..., N do tamanho das amostras coletadas
santos_tempo = np.arange(0, santos_covid_casos.size, 1)
# número de amostras
N_santos = santos_tempo.size
#
print("Dados de casos confirmados - Santos")
santos_covid_df.describe()

Dados de casos confirmados - Santos


,casos,obitos,pop
count,370.000000,370.000000,370.0
mean,17246.051351,503.732432,428703.0
std,13001.072447,353.662251,0.0
min,0.000000,0.000000,428703.0
25%,2791.500000,120.500000,428703.0
50%,19346.000000,563.000000,428703.0
75%,27429.500000,781.000000,428703.0
max,39910.000000,1105.000000,428703.0


# Ajustando o modelo de Covid-19 aos dados públicos
O objetivo é ajustar os parâmetros do modelo:
\begin{align}
\begin{split}
\dfrac{ds}{dt}&=\mu+\gamma -\alpha(1-\theta)si-(\mu+\gamma) s -\gamma i -\gamma s_{ick}\\
%
\dfrac{di}{dt}&=\alpha(1-\theta)si-(\beta_1+\beta_2+\mu)i\\
\dfrac{ds_{ick}}{dt}&=\beta_2i-(\beta_3+\mu)s_{ick}
\end{split}
\label{eq:constant-perc-pop-reduced-model}
\end{align}
com o índice de isolamento $\theta$ assumindo diferentes funções:
1. $\theta_1$ uma constante igual á média aritmética dos dados públicos;
2. $\theta_2(t)=at+b$, uma reta de tendência;
3. $\theta_3(t)=a_9t^9+\cdots+a_1t+a_0$, Um polinômio de ordem 9.

Para modelar a curva de casos confirmados de COVID-19 nas três cidades.

09/03/202: Após várias tentativas obteve sucesso com um fit de segunda ordem: $\theta(t)=a_2t^2+a_1t+a_0$

---

Funções a serem utilizadas em todos os casos.

In [9]:
def rhs(t,x,*p):
    # O lado direito da função    
    # ds/dt = mu + gamma - alpha(1-theta)*s*i - (mu + gamma)*s - gamma*i - gamma*sick
    # di/dt = alpha*(1-theta)*s*i - (beta1 + beta2 + mu)*i
    # dsick/dt = beta2*i - (beta3 + mu)* sick
    #
    # this is the right-hand side funtion, implemented for numertical integration
    
    # verificando número de argumentos
    if len(p)!=7:
        raise Exception("Erro no número de parâmetros!")
    s = x[0]
    i = x[1]
    sick = x[2]
    #
    mu = p[0]
    gamma = p[1]
    alpha = p[2]
    theta = p[3]
    beta1 = p[4]
    beta2 = p[5]
    beta3 = p[6]
    return np.array([mu + gamma - alpha*(1-theta(t))*s*i - (mu+gamma)*s - gamma*i - gamma*sick,
                     alpha*(1-theta(t))*s*i - (beta1+beta2+mu)*i,
                     beta2*i - (beta3+mu)*sick])      

# A função de chamada do optimizador
def f(x, gamma, alpha, beta1, beta2, beta3, s0, theta, mu, N):
        # in this version I adjust the initial condition so s0 + i0 + r0 = 1
        i0 = 1-s0
        r0 = 0
        x0 = np.array([s0,i0,r0]) # initial condition
        t0 = 0 # simulação comeza no dia 0
        tf = N-1 # até N-1 dias
        h = 1 # o paso de integração é um dia
        t,sol = rk4(lambda t, x: rhs(t, x, mu, gamma, alpha, theta, beta1, beta2, beta3), x0, t0, tf, h)
        return sol[:, 2]

def otimiza(params_min, params_max, p0, theta, mu, N, dados_tempo, dados_covid_casos, dados_tot_pop):
    # Parâmetros para optimização
    sigma = None
    absolute_sigma = False
    # checar se ha algum NaN ou InF nos dados
    check_finite = True
    # os valores mínimos que os parâmetros podem tomar
    #params_min = [gamma_min, alpha_min, beta1_min, beta2_min, beta3_min, s0_min]
    # os valores máximos que os parâmetros podem tomar
    #params_max = [gamma_max, alpha_max, beta1_max, beta2_max, beta3_max, s0_max]
    # organizando os vetores dos limites inferirior e superior
    bounds = (params_min, params_max)
    # método de otimização (escoler um deles)
    # ‘dogbox’ : dogleg algorithm with rectangular trust regions,
    # 'trf' : Trust Region Reflective algorithm
    method = 'trf'
    # Jacobiano
    jac = None
    # chute inicial para os parâmetros, para ajustar o resultado da simulação podemos ajustar esses chutes iniciais
    #p0 = np.array([gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0])
    # a otimização
    print('\nRunning the optimization ...')
    f_adj = lambda x, gamma, alpha, beta1, beta2, beta3, s0: f(x, gamma, alpha, beta1, beta2, beta3, s0, theta, mu, N)
    popt, pvoc = optimization.curve_fit(f_adj, dados_tempo, dados_covid_casos/dados_tot_pop, p0, sigma, absolute_sigma, check_finite, bounds,method, jac)
    return(popt, pvoc)

# Ajuste de Parâmetros

In [10]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# parâmetros para ajustar o tamanho das figuras
#plt.rcParams['figure.figsize'] = [15, 5] # <<<<<-------

## Ajuste de parâmetros São Paulo

In [11]:
# Ajuste do índice de isolamento de São Paulo
def isol_saopaulo(poly_order, gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0 ):
    #
    X_saopaulo = np.arange(0,saopaulo_isol_df['Dados'].size, 1)
    #
    coefs, stats = poly.polyfit(X_saopaulo, saopaulo_isol_df['Dados'].to_numpy(), poly_order , full=True)
    ffit_saopaulo = np.polynomial.Polynomial(coefs)
    #
    mu_saopaulo = 3.595e-05 # (Aqui estou utilizando o dado utilizado no artigo anterior) (utilizei 2e-5 nas simulações anteriores)
    
    p0 = [gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0]
      
    popt_1, pvoc_1 = otimiza(pmin, pmax, p0, ffit_saopaulo, mu_saopaulo, N_saopaulo, saopaulo_tempo, saopaulo_covid_casos, saopaulo_tot_pop)

    perr = np.sqrt(np.diag(pvoc_1))
    print('Standard deviation errors on the parameters = ', perr)
    print('\nParameters:')
    print('-----------')
    print('mu =', mu_saopaulo)
    print('theta =', ffit_saopaulo)
    print('Fitted parameters:')
    gamma_opt = popt_1[0]
    print('gamma =', gamma_opt)
    alpha_opt = popt_1[1]
    print('alpha =', alpha_opt)
    beta1_opt = popt_1[2]
    print('beta1 =', beta1_opt)
    beta2_opt = popt_1[3]
    print('beta2 =', beta2_opt)
    beta3_opt = popt_1[4]
    print('beta3 =', beta3_opt)
    s0_opt = popt_1[5]
    print('s0 =', s0_opt)
    i0_opt = 1-s0_opt
    print('i0 =', i0_opt)
    # initial condition
    x0 = np.array([s0_opt, i0_opt, 0])
    # parameters
    t0 = 0
    tf = N_saopaulo-1
    h = 1
    #    
    # rodando Runge-Kutta de 4ta ordem de passo fixo
    t, sol =  rk4(lambda t, x: rhs(t, x, mu_saopaulo, gamma_opt, alpha_opt, ffit_saopaulo, beta1_opt, beta2_opt, beta3_opt), x0, t0, tf, h)
    # o gráfico do ajuste
    fig1, ax1 = plt.subplots(figsize=(fig_width, fig_height), dpi=fig_dpi)
    mloc = MonthLocator()
        
    ax1.scatter(saopaulo_isol_df['Data'],saopaulo_isol_df['Dados'].to_numpy(), color='gray', label='Real data')
    ax1.plot(saopaulo_isol_df['Data'],ffit_saopaulo(X_saopaulo), linewidth=3, label='2nd order polynomial fitting')
    ax1.format_xdata = mdates.DateFormatter('%y-%m')
    ax1.xaxis.set_major_locator(MonthLocator())
    ax1.xaxis.set_tick_params(rotation=45)
    
    ax1.legend()
    ax1.grid(which='major')
    ax1.set_ylabel('Isolation index in São Paulo city')
      
    #plt.savefig('figures/isolamento_saopaulo.eps', format='eps')

    plt.show()
    
    fig2, ax2 = plt.subplots(figsize=(fig_width, fig_height), dpi=fig_dpi)
    #decimar os dados reais para melhor visualização
    n = 5 # fração 1/n de dados
    j = n*np.array(np.arange(int(saopaulo_dias.size/n)))
    ax2.scatter(saopaulo_dias[j],saopaulo_covid_casos[j], label='Real data', color='tab:orange')
    
    #ax2.scatter(saopaulo_dias, saopaulo_covid_casos, linewidth=3, label='Dados reais - São Paulo', color='tab:orange')
    ax2.plot(saopaulo_dias, sol[:,2]*saopaulo_tot_pop, linewidth=3, label='Fitting', color='tab:blue')
    ax2.ticklabel_format(axis='y', style='sci',scilimits=(4,5))
    ax2.xaxis.set_tick_params(rotation=45)
    ax2.grid()
    ax2.legend()
    #ax2.set_title('Parametric fitting')
    #ax2.set_xlabel('Dates')
    ax2.set_ylabel('COVID-19 confirmed cases in São Paulo city')
    
    #plt.savefig('figures/casos_confirmados_saopaulo.eps', format='eps')
    
    plt.show()
    
    # salvando dados em arquivo
    parametros_otimos = {
        'Parametros':["$\\mu$", '$\\gamma$', '$\\alpha$', '$\\theta$', '$\\beta_1$', '$\\beta_2$', '$\\beta_3$', '$s_0$', '$i_0$'],\
        'Valor':[mu_saopaulo, gamma_opt, alpha_opt, saopaulo_isol_df.Dados.mean() ,beta1_opt, beta2_opt, beta3_opt, s0_opt, i0_opt]}
    saopaulo_parametros_df = pd.DataFrame(data=parametros_otimos, columns= ['Parametros', 'Valor'])
    
    with open('saopaulo_parameters_table.tex', 'w') as tf:
        tf.write(saopaulo_parametros_df.to_latex(index=False, escape=False))
    
    saopaulo_parametros_df.to_csv('saopaulo_param_otim.csv', index=False)
    
    # adicionalmente salvamos a simulação obtida.
    simul = np.concatenate((t.reshape(-1,1),sol),axis=1)
    simul_df = pd.DataFrame(data=simul, columns=['tempo', 's', 'i', 'sick'])
    simul_df.to_csv('saopaulo_simula_otim.csv', index=False)
      
    return

# os valores mínimos que os parâmetros podem tomar
gamma_min = 0.01
alpha_min = 0.1
beta1_min = 0.0222
beta2_min = 0.048
beta3_min = 0.0222
s0_min = 0
# organizando os parâmetros
pmin = [gamma_min, alpha_min, beta1_min, beta2_min, beta3_min, s0_min]
  
gamma_max = 0.1
alpha_max = 1
beta1_max = 0.2
beta2_max = 0.2
beta3_max = 0.067
s0_max    = 0.9999
   
pmax = [gamma_max, alpha_max, beta1_max, beta2_max, beta3_max, s0_max]
   
# organizando os parâmetros 

poly_order_sld_1 = widgets.IntSlider(
                 min = 0,
                 max = 12,
                 step= 1,
                 description = 'Ordem do pol.:',
                 value = 2)

gamma_0_sld_1 = widgets.FloatSlider(
              min = gamma_min,
              max = gamma_max,
              step= (gamma_max-gamma_min)/10,
              description = 'Gamma_0:',
              value = (gamma_max-gamma_min)/2)

alpha_0_sld_1 = widgets.FloatSlider(
              min = alpha_min,
              max = alpha_max,
              step= (alpha_max-alpha_min)/10,
              description = 'Alpha_0:',
              value = (alpha_max-alpha_min)/2)

beta1_0_sld_1 = widgets.FloatSlider(
              min = beta1_min,
              max = beta1_max,
              step= (beta1_max-beta1_min)/10,
              description = 'Beta1_0:',
              value = (beta1_max-beta1_min)/2)

beta2_0_sld_1 = widgets.FloatSlider(
              min = beta2_min,
              max = beta2_max,
              step= (beta2_max-beta2_min)/10,
              description = 'Beta2_0:',
              value = (beta2_max-beta2_min)/2)

beta3_0_sld_1 = widgets.FloatSlider(
              min = beta3_min,
              max = beta3_max,
              step= (beta3_max-beta3_min)/10,
              description = 'Beta3_0:',
              value = (beta3_max-beta3_min)/2)

s0_0_sld_1 = widgets.FloatSlider(
              min = s0_min,
              max = s0_max,
              step= (s0_max-s0_min)/10,
              description = 's0_0:',
              value = s0_max)

interact(isol_saopaulo, poly_order = poly_order_sld_1,
               gamma_0 = gamma_0_sld_1,
               alpha_0 = alpha_0_sld_1, 
               beta1_0 = beta1_0_sld_1, 
               beta2_0 = beta2_0_sld_1, 
               beta3_0 = beta3_0_sld_1, 
               s0_0 = s0_0_sld_1)

interactive(children=(IntSlider(value=2, description='Ordem do pol.:', max=12), FloatSlider(value=0.0450000000…

<function __main__.isol_saopaulo(poly_order, gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0)>

## Ajuste de parâmetros Campinas

In [12]:
# Ajuste do índice de isolamento de Campinas
def isol_campinas(poly_order, gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0 ):
    #
    ma_dias = 21 # média móvel de X dias
    ma_campinas_isol = campinas_isol_df['Dados'].rolling(ma_dias, min_periods=1).mean()
    #
    X_campinas = np.arange(0, ma_campinas_isol.size, 1)
    #X_campinas = np.arange(0,campinas_isol_df['Dados'].size, 1)
    #coefs, stats = poly.polyfit(X_campinas, campinas_isol_df['Dados'].to_numpy(), poly_order , full=True)
    coefs, stats = poly.polyfit(X_campinas, ma_campinas_isol.to_numpy(), poly_order , full=True)
    
    ffit_campinas = np.polynomial.Polynomial(coefs)
    #
    mu_campinas = 3.353e-05 # (Aqui estou utilizando o dado utilizado no artigo anterior) (utilizei 2e-5 nas simulações anteriores)
    
    p0 = [gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0]
      
    popt_1, pvoc_1 = otimiza(pmin, pmax, p0, ffit_campinas, mu_campinas, N_campinas, campinas_tempo, campinas_covid_casos, campinas_tot_pop)

    perr = np.sqrt(np.diag(pvoc_1))
    print('Standard deviation errors on the parameters = ', perr)
    print('\nParameters:')
    print('-----------')
    print('mu =', mu_campinas)
    print('theta =', ffit_campinas)
    print('Fitted parameters:')
    gamma_opt = popt_1[0]
    print('gamma =', gamma_opt)
    alpha_opt = popt_1[1]
    print('alpha =', alpha_opt)
    beta1_opt = popt_1[2]
    print('beta1 =', beta1_opt)
    beta2_opt = popt_1[3]
    print('beta2 =', beta2_opt)
    beta3_opt = popt_1[4]
    print('beta3 =', beta3_opt)
    s0_opt = popt_1[5]
    print('s0 =', s0_opt)
    i0_opt = 1-s0_opt
    print('i0 =', i0_opt)
    # initial condition
    x0 = np.array([s0_opt, i0_opt, 0])
    # parameters
    t0 = 0
    tf = N_campinas-1
    h = 1
    #    
    # rodando Runge-Kutta de 4ta ordem de passo fixo
    t, sol =  rk4(lambda t, x: rhs(t, x, mu_campinas, gamma_opt, alpha_opt, ffit_campinas, beta1_opt, beta2_opt, beta3_opt), x0, t0, tf, h)
    # o gráfico do ajuste
    
    fig1, ax1 = plt.subplots(figsize=(fig_width, fig_height), dpi=fig_dpi)
    ax1.plot(campinas_isol_df['Data'], ffit_campinas(X_campinas), linewidth=3, label='2nd order polynomial fitting', color='tab:blue')
    ax1.plot(campinas_isol_df['Data'], ma_campinas_isol.to_numpy(), color='tab:orange', linewidth=3 ,label='Moving average - '+str(ma_dias)+' days')
    ax1.scatter(campinas_isol_df['Data'], campinas_isol_df['Dados'].to_numpy(), color='gray', label='Real data')
    
    ax1.format_xdata = mdates.DateFormatter('%y-%m')
    ax1.xaxis.set_major_locator(MonthLocator())
    ax1.xaxis.set_tick_params(rotation=45)
    
    ax1.legend()
    ax1.grid(which='major')
    ax1.set_ylabel('Isolation index in Campinas city')
    
    #plt.savefig('figures/isolamento_campinas.eps', format='eps')
    plt.show()
    
    fig2, ax2 = plt.subplots(figsize=(fig_width, fig_height), dpi=fig_dpi)
    #decimar os dados reais para melhor visualização
    n = 5 # fração 1/n de dados
    j = n*np.array(np.arange(int(campinas_dias.size/n)))
    
    ax2.scatter(campinas_dias[j], campinas_covid_casos[j], linewidth=3, label='Real data', color='tab:orange')
    ax2.plot(campinas_dias, sol[:,2]*campinas_tot_pop, linewidth=3, label='Fitting', color='tab:blue')
    ax2.ticklabel_format(axis='y', style='sci',scilimits=(3,4))
    ax2.xaxis.set_tick_params(rotation=45)
    ax2.grid()
    ax2.legend()
    #ax2.set_title('Parametric fitting')
    #ax2.set_xlabel('Dates')
    ax2.set_ylabel('COVID-19 confirmed cases in Campinas city')
    
    #plt.savefig('figures/casos_confirmados_campinas.eps', format='eps')
    plt.show()
    
    # salvando dados em arquivo
    parametros_otimos = {
        'Parametros':['$\mu$', '$\gamma$', '$\alpha$', '$\theta$', '$\beta_1$', '$\beta_2$', '$\beta_3$', '$s_0$', '$i_0$'],\
        'Valor':[mu_campinas, gamma_opt, alpha_opt, campinas_isol_df.Dados.mean() ,beta1_opt, beta2_opt, beta3_opt, s0_opt, i0_opt]}
    campinas_parametros_df = pd.DataFrame(data=parametros_otimos, columns= ['Parametros', 'Valor'])
    
    with open('campinas_parameters_table.tex', 'w') as tf:
        tf.write(campinas_parametros_df.to_latex(index=False, escape=False))
        
    campinas_parametros_df.to_csv('campinas_param_otim.csv', index=False)
    
    # adicionalmente salvamos a simulação obtida.
    simul = np.concatenate((t.reshape(-1,1),sol),axis=1)
    simul_df = pd.DataFrame(data=simul, columns=['tempo', 's', 'i', 'sick'])
    simul_df.to_csv('campinas_simula_otim.csv', index=False)
    
    return

# os valores mínimos que os parâmetros podem tomar
gamma_min = 0.01
alpha_min = 0.1
beta1_min = 0.0222
beta2_min = 0.048
beta3_min = 0.0222
s0_min = 0
# organizando os parâmetros
pmin = [gamma_min, alpha_min, beta1_min, beta2_min, beta3_min, s0_min]
  
gamma_max = 0.1
alpha_max = 1
beta1_max = 0.2
beta2_max = 0.2
beta3_max = 0.067
s0_max    = 0.9999
   
pmax = [gamma_max, alpha_max, beta1_max, beta2_max, beta3_max, s0_max]
   
# organizando os parâmetros 

poly_order_sld_2 = widgets.IntSlider(
                 min = 0,
                 max = 12,
                 step= 1,
                 description = 'Ordem do pol.:',
                 value = 2)

gamma_0_sld_2 = widgets.FloatSlider(
              min = gamma_min,
              max = gamma_max,
              step= (gamma_max-gamma_min)/10,
              description = 'Gamma_0:',
              value = (gamma_max-gamma_min)/2)

alpha_0_sld_2 = widgets.FloatSlider(
              min = alpha_min,
              max = alpha_max,
              step= (alpha_max-alpha_min)/10,
              description = 'Alpha_0:',
              value = (alpha_max-alpha_min)/2)

beta1_0_sld_2 = widgets.FloatSlider(
              min = beta1_min,
              max = beta1_max,
              step= (beta1_max-beta1_min)/10,
              description = 'Beta1_0:',
              value = (beta1_max-beta1_min)/2)

beta2_0_sld_2 = widgets.FloatSlider(
              min = beta2_min,
              max = beta2_max,
              step= (beta2_max-beta2_min)/10,
              description = 'Beta2_0:',
              value = (beta2_max-beta2_min)/2)

beta3_0_sld_2 = widgets.FloatSlider(
              min = beta3_min,
              max = beta3_max,
              step= (beta3_max-beta3_min)/10,
              description = 'Beta3_0:',
              value = (beta3_max-beta3_min)/2)

s0_0_sld_2 = widgets.FloatSlider(
              min = s0_min,
              max = s0_max,
              step= (s0_max-s0_min)/10,
              description = 's0_0:',
              value = s0_max)

interact(isol_campinas, poly_order = poly_order_sld_2,
               gamma_0 = gamma_0_sld_2,
               alpha_0 = alpha_0_sld_2, 
               beta1_0 = beta1_0_sld_2, 
               beta2_0 = beta2_0_sld_2, 
               beta3_0 = beta3_0_sld_2, 
               s0_0 = s0_0_sld_2)

interactive(children=(IntSlider(value=2, description='Ordem do pol.:', max=12), FloatSlider(value=0.0450000000…

<function __main__.isol_campinas(poly_order, gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0)>

## Ajuste de parâmetros Santos

In [13]:
# Ajuste do índice de isolamento de São Paulo
def isol_santos(poly_order, gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0 ):
    #
    ma_dias = 21 # média móvel de x dias
    ma_santos_isol = santos_isol_df['Dados'].rolling(ma_dias, min_periods=1).mean()
    #
    X_santos = np.arange(0, ma_santos_isol.size, 1)
    #X_campinas = np.arange(0,campinas_isol_df['Dados'].size, 1)
    #coefs, stats = poly.polyfit(X_campinas, campinas_isol_df['Dados'].to_numpy(), poly_order , full=True)
    coefs, stats = poly.polyfit(X_santos, ma_santos_isol.to_numpy(), poly_order , full=True)
    #X_santos = np.arange(0, santos_isol_df['Dados'].size, 1)
    #
    #coefs, stats = poly.polyfit(X_santos, santos_isol_df['Dados'].to_numpy(), poly_order , full=True)
    
    ffit_santos = np.polynomial.Polynomial(coefs)
    #
    mu_santos = 2.693e-05 # (Aqui estou utilizando o dado utilizado no artigo anterior) (utilizei 2e-5 nas simulações anteriores)
    
    p0 = [gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0]
      
    popt_1, pvoc_1 = otimiza(pmin, pmax, p0, ffit_santos, mu_santos, N_santos, santos_tempo, santos_covid_casos, santos_tot_pop)

    perr = np.sqrt(np.diag(pvoc_1))
    print('Standard deviation errors on the parameters = ', perr)
    print('\nParameters:')
    print('-----------')
    print('mu =', mu_santos)
    print('theta =', ffit_santos)
    print('Fitted parameters:')
    gamma_opt = popt_1[0]
    print('gamma =', gamma_opt)
    alpha_opt = popt_1[1]
    print('alpha =', alpha_opt)
    beta1_opt = popt_1[2]
    print('beta1 =', beta1_opt)
    beta2_opt = popt_1[3]
    print('beta2 =', beta2_opt)
    beta3_opt = popt_1[4]
    print('beta3 =', beta3_opt)
    s0_opt = popt_1[5]
    print('s0 =', s0_opt)
    i0_opt = 1-s0_opt
    print('i0 =', i0_opt)
    # initial condition
    x0 = np.array([s0_opt, i0_opt, 0])
    # parameters
    t0 = 0
    tf = N_santos-1
    h = 1
    #    
    # rodando Runge-Kutta de 4ta ordem de passo fixo
    t, sol =  rk4(lambda t, x: rhs(t, x, mu_santos, gamma_opt, alpha_opt, ffit_santos, beta1_opt, beta2_opt, beta3_opt), x0, t0, tf, h)
    
    fig1, ax1 = plt.subplots(figsize=(fig_width, fig_height), dpi=fig_dpi)
    ax1.plot(santos_isol_df['Data'], ffit_santos(X_santos), linewidth=3, label='2nd order polynomial fitting', color='tab:blue')
    ax1.plot(santos_isol_df['Data'], ma_santos_isol.to_numpy(), color='tab:orange', linewidth=3 ,label='Moving average - '+str(ma_dias)+' days')
    ax1.scatter(santos_isol_df['Data'], santos_isol_df['Dados'].to_numpy(), color='gray', label='Real data')
    
    ax1.format_xdata = mdates.DateFormatter('%y-%m')
    ax1.xaxis.set_major_locator(MonthLocator())
    ax1.xaxis.set_tick_params(rotation=45)
    
    ax1.legend()
    ax1.grid(which='major')
    ax1.set_ylabel('Isolation index Santos city')
    
    #plt.savefig('figures/isolamento_santos.eps', format='eps')
    plt.show()
    
    fig2, ax2 =plt.subplots(figsize=(fig_width, fig_height), dpi=fig_dpi)
    #decimar os dados reais para melhor visualização
    n = 5 # fração 1/n de dados
    j = n*np.array(np.arange(int(santos_dias.size/n)))
    
    ax2.scatter(santos_dias[j], santos_covid_casos[j], linewidth=3, label='Real data', color='tab:orange')
    ax2.plot(santos_dias, sol[:,2]*santos_tot_pop, linewidth=3, label='Fitting', color='tab:blue')
    ax2.ticklabel_format(axis='y', style='sci',scilimits=(3,4))
    ax2.xaxis.set_tick_params(rotation=45)
    ax2.grid()
    ax2.legend()
    #ax2.set_title('Parametric fitting')
    #ax2.set_xlabel('Dates')
    ax2.set_ylabel('COVID-19 confirmed cases in Santos city')
    
    #plt.savefig('figures/casos_confirmados_santos.eps', format='eps')
    plt.show()
    
    
    # salvando dados em arquivo
    parametros_otimos = {
        'Parametros':['$\mu$', '$\gamma$', '$\alpha$', '$\theta$', '$\beta_1$', '$\beta_2$', '$\beta_3$', '$s_0$', '$i_0$'],\
        'Valor':[mu_santos, gamma_opt, alpha_opt, santos_isol_df.Dados.mean() ,beta1_opt, beta2_opt, beta3_opt, s0_opt, i0_opt]}
    santos_parametros_df = pd.DataFrame(data=parametros_otimos, columns= ['Parametros', 'Valor'])
    
    with open('santos_parameters_table.tex', 'w') as tf:
        tf.write(santos_parametros_df.to_latex(index=False, escape=False))
    
    santos_parametros_df.to_csv('santos_param_otim.csv', index=False)
    
    # adicionalmente salvamos a simulação obtida.
    simul = np.concatenate((t.reshape(-1,1),sol),axis=1)
    simul_df = pd.DataFrame(data=simul, columns=['tempo', 's', 'i', 'sick'])
    simul_df.to_csv('santos_simula_otim.csv', index=False)
    
    return

# os valores mínimos que os parâmetros podem tomar
gamma_min = 0.01
alpha_min = 0.1
beta1_min = 0.0222
beta2_min = 0.048
beta3_min = 0.0222
s0_min = 0
# organizando os parâmetros
pmin = [gamma_min, alpha_min, beta1_min, beta2_min, beta3_min, s0_min]
  
gamma_max = 0.1
alpha_max = 1
beta1_max = 0.2
beta2_max = 0.2
beta3_max = 0.067
s0_max    = 0.9999
   
pmax = [gamma_max, alpha_max, beta1_max, beta2_max, beta3_max, s0_max]
   
# organizando os parâmetros 

poly_order_sld_3 = widgets.IntSlider(
                 min = 0,
                 max = 12,
                 step= 1,
                 description = 'Ordem do pol.:',
                 value = 2)

gamma_0_sld_3 = widgets.FloatSlider(
              min = gamma_min,
              max = gamma_max,
              step= (gamma_max-gamma_min)/10,
              description = 'Gamma_0:',
              value = (gamma_max-gamma_min)/2)

alpha_0_sld_3 = widgets.FloatSlider(
              min = alpha_min,
              max = alpha_max,
              step= (alpha_max-alpha_min)/10,
              description = 'Alpha_0:',
              value = (alpha_max-alpha_min)/2)

beta1_0_sld_3 = widgets.FloatSlider(
              min = beta1_min,
              max = beta1_max,
              step= (beta1_max-beta1_min)/10,
              description = 'Beta1_0:',
              value = (beta1_max-beta1_min)/2)

beta2_0_sld_3 = widgets.FloatSlider(
              min = beta2_min,
              max = beta2_max,
              step= (beta2_max-beta2_min)/10,
              description = 'Beta2_0:',
              value = (beta2_max-beta2_min)/2)

beta3_0_sld_3 = widgets.FloatSlider(
              min = beta3_min,
              max = beta3_max,
              step= (beta3_max-beta3_min)/10,
              description = 'Beta3_0:',
              value = (beta3_max-beta3_min)/2)

s0_0_sld_3 = widgets.FloatSlider(
              min = s0_min,
              max = s0_max,
              step= (s0_max-s0_min)/10,
              description = 's0_0:',
              value = s0_max)

interact(isol_santos, poly_order = poly_order_sld_3,
               gamma_0 = gamma_0_sld_3,
               alpha_0 = alpha_0_sld_3, 
               beta1_0 = beta1_0_sld_3, 
               beta2_0 = beta2_0_sld_3, 
               beta3_0 = beta3_0_sld_3, 
               s0_0 = s0_0_sld_3)

interactive(children=(IntSlider(value=2, description='Ordem do pol.:', max=12), FloatSlider(value=0.0450000000…

<function __main__.isol_santos(poly_order, gamma_0, alpha_0, beta1_0, beta2_0, beta3_0, s0_0)>